<a href="https://colab.research.google.com/github/Droushb/NULP_NLP/blob/main/LPNLP06_Spellchecker_quest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spellchecker quest

Хтось наробив помилок у віршах Тараса Шевченка. Наша задача -- виправити ці помилки і прочитати приховане повідомлення.

## Задача

Ви отримаєте тренувальні та тестувальні дані.

Тренувальні дані знаходяться в полі `lab.train_text`. Це звичайний нерозмічений текст. На ньому необхідно натренувати мовну модель. Підійде будь-яка. Я би радив feed-forward нейрону модель з токенізацією по літерах, бо це те, що ми проходили на останній лекції. Але n-грамна теж має спрацювати.

Тестувальні дані знаходиться в полі `lab.test_items`. Приклад одного елемента:

```json
{
  "text": "Співали б прозу, та по ножах,",
  "error_start": 23,
  "error_end": 28,
  "error": "ножах",
  "corrections": [
    "ногах",
    "йотах",
    "єнотах",
    "ножах",
    "нотах"
  ]
}
```

`error_start` та `error_end` вказують на місцезнаходження помилки в тексті (в символах). У данному прикладі, помилкою є `text[23:28]`, тобто слово "ножах".

`corrections` -- це список можливих виправлень.

Ваша задача -- обрати правильне виправлення серед запропонованих.


## Приховане повідомлення

Один приклад в `lab.test_items` дає можливість прочитати одну літеру прихованого повідомлення. Для цього знайдіть різницю між літерами слова з помилкою (`error`) та обраним виправленням. Надрукуйте цю літеру. Якщо слово з помилкою направді правильне, а таке теж буває, надрукуйте пробіл. Приклади:

```
Error               Correction     To print
-------------------------------------------
привіт               приліт        л
пні                  поні          о
баллет               балет         л
привіт               привіт        (space)
```

Приховане повідомлення, яке ви побачите в результаті це рядок з віршу одного з українських авторів.

Відповідь на квест -- ім'я автора/ки у форматі "Ім'я Прізвище".

Полетіли! 🚀

In [ ]:
!pip install --quiet --ignore-installed http://nlp.band/static/pypy/lpnlp-2023.10.2-py3-none-any.whl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 9.1 MB/s eta 0:00:00


In [ ]:
import lpnlp

lab = lpnlp.start(
    email="bohdan.drushkevych.kn.2021@lpnu.ua",                   # <----------- Заповніть це поле
    lab="quest_spellchecker"
    )

Удачі!


## Мовна модель

Натренуйте свою мовну модель тут

In [ ]:
print(lab.train_text[:330])

﻿ПРИЧИННА

Реве та стогне Дніпр широкий,
Сердитий вітер завива,
Додолу верби гне високі,
Горами хвилю підійма.
І блідий місяць на ту пору
Із хмари де-де виглядав,
Неначе човен в синім морі,
То виринав, то потопав.
Ще треті півні не співали,
Ніхто нігде не гомонів,
Сичі в гаю перекликались,
Та ясен раз у раз скрипів.


In [ ]:
import torch
from torch import nn
from typing import Iterable, List, Tuple, Dict, Union
from tqdm import tqdm
import collections

In [ ]:
class Vocabulary:
    def __init__(self, tokens, unk_token="<UNK>", min_count=None):
        if min_count is None:
            min_count = 0

        count = collections.Counter(tokens)
        tokens = [w for w, c in count.most_common() if c >= min_count]

        self._itos = list(tokens)
        self.unk_token = unk_token
        try:
            self.unk_index = self._itos.index(unk_token)
        except ValueError:
            self._itos.insert(0, unk_token)
            self.unk_index = 0

        self._stoi = {token: index for index, token in enumerate(self._itos)}

    def stoi(self, token: str) -> int:
        return self._stoi.get(token, self.unk_index)

    def itos(self, index: int) -> str:
        return self._itos[index]

    def __getitem__(self, key):
        if isinstance(key, str):
            return self.stoi(key)
        elif isinstance(key, int):
            return self.itos(key)
        raise TypeError("Unsupported key type")

    def get(self, key, default=None):
        return self._stoi.get(key, default)

    @property
    def tokens(self):
        return self._itos

    def __len__(self) -> int:
        return len(self._itos)

In [ ]:
class BengioLmModel(nn.Module):
    def __init__(
          self,
          vocab_size: int,
          embed_dim: int,
          context_len: int,
          hidden_dim: int
        ) -> None:

        super().__init__()

        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.context_len = context_len
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.W = nn.Linear(context_len * embed_dim, hidden_dim)
        self.U = nn.Linear(hidden_dim, vocab_size)

    def forward(self, X_indexes: torch.tensor):
        X = self.embed(X_indexes)
        e = X.view(-1, self.context_len * self.embed_dim)

        h = torch.tanh(self.W(e))
        logits = self.U(h)

        log_probs = torch.log_softmax(logits, dim=-1)

        return log_probs

In [ ]:
def prepare_data(tokens: List[str], context_len: int):
    for i in range(context_len, len(tokens)):
        context = tokens[i - context_len:i]
        target = tokens[i]

        yield context, target

def vectorize(tokens: Iterable[str], vocab: Vocabulary) -> torch.tensor:
   X = torch.tensor([vocab.stoi(token) for token in tokens])
   return X

def tokenize(text: str) -> List[str]:
   return list(text.lower())

def batch_it(xs, batch_size):
    batch = []

    for x in xs:
        batch.append(x)
        if len(batch) == batch_size:
            yield batch
            batch = []

    if batch:
        yield batch

In [ ]:
def train(
    model: nn.Module,
    vocabulary: Vocabulary,
    parms: Dict[str, Union[float, int]],
    text_tokens: List[str]
):

  optimizer = torch.optim.Adam(model.parameters(), lr=hparams["learning_rate"])
  loss_fn = nn.NLLLoss()

  for epoch in range(hparams["num_epochs"]):
      total_loss = 0.0
      examples = prepare_data(train_text_tokens, hparams["context_len"])
      examples = list(examples)

      for batch in tqdm(batch_it(examples, batch_size=hparams["batch_size"]), leave=False):
          X_batch = []
          y_batch = []
          for context, target in batch:
            X = vectorize(context, vocab)
            y = vectorize([target], vocab)
            X_batch.append(X)
            y_batch.append(y)

          X_batch = torch.stack(X_batch)
          y_batch = torch.tensor(y_batch)

          optimizer.zero_grad()

          log_probs = model(X)
          loss = loss_fn(log_probs, y)
          loss.backward()
          optimizer.step()
          total_loss += loss.sum().item()

      print(f" Epoch: {epoch + 1} Loss: {total_loss / len(examples)}")

In [ ]:
train()

 Epoch: 1 Loss: 0.002527301924407955


 Epoch: 2 Loss: 0.0011652723458189056


 Epoch: 3 Loss: 0.0007886722958323252


 Epoch: 4 Loss: 0.0005114673151130026


 Epoch: 5 Loss: 0.0003140018146722566


 Epoch: 6 Loss: 0.00019170424824809068


 Epoch: 7 Loss: 0.00012255399624734324


 Epoch: 8 Loss: 8.419478172763686e-05


 Epoch: 9 Loss: 6.148407479938844e-05


 Epoch: 10 Loss: 4.771078017946218e-05


In [ ]:
train_text_tokens = tokenize(lab.train_text)
vocab = Vocabulary(train_text_tokens)

hparams: Dict[str, Union[float, int]] = {
    "vocab_size": len(vocab),
    "embed_dim": 64,
    "context_len": 4,
    "hidden_dim": 128,
    "learning_rate": 0.001,
    "num_epochs": 10,
    "batch_size": 512
}

model = BengioLmModel(
    vocab_size=hparams.get("vocab_size"),
    embed_dim=hparams.get("embed_dim"),
    context_len=hparams.get("context_len"),
    hidden_dim=hparams.get("hidden_dim")
)

train(model, vocab, hparams, train_text_tokens)

cpu


 Epoch: 1 Loss: 0.00465836627009413


 Epoch: 2 Loss: 0.004109407542358831


 Epoch: 3 Loss: 0.003940937720011687


 Epoch: 4 Loss: 0.003843840562051609


...

 Epoch: 6 Loss: 0.00373220545104013


 Epoch: 7 Loss: 0.003696935380883881


 Epoch: 8 Loss: 0.00366926525227913


 Epoch: 9 Loss: 0.0036467229242251897


 Epoch: 10 Loss: 0.0036277734706472867


# Читаємо між рядків

In [ ]:
import math
import collections
from typing import List, Tuple


# Допоміжна фунція:
def get_letter(w1: str, w2: str) -> str:
    """Повертає літеру, якої відрізняються слова або пробіл для однакових слів.
    """

    letters1 = collections.Counter(w1)
    letters2 = collections.Counter(w2)

    diff = letters1 - letters2
    if len(diff) != 1:
        return " "

    return diff.most_common()[0][0]


def score_text(text: str, model, vocab) -> float:
    """Повертає perplexity або log-probability для тексту. """

    tokens = tokenize(text)

    total_log_prob = 0.0

    for context, target in prepare_data(tokens, model.context_len):
        X = vectorize(context, vocab)
        target = vectorize([target], vocab)[0]
        log_probs = model(X)
        target_log_prob = log_probs[0, target]
        total_log_prob += target_log_prob

    return torch.exp(-total_log_prob / len(tokens)).item()


# Можете змінювати параметри та весь цей код, якщо потрібно
def solve(model, vocab, test_items) -> Tuple[List[str], str]:
    """Повертає список виправлених слів для кожного з текстів в test_items та
    секретне повідомлення.
    """

    choices = []
    secret = []

    for item in test_items:
        scores = []
        for corr in item['corrections']:

            # Підставляємо слово-кандидат в текст
            text = item['text'][:item['error_start']] + corr + item['text'][item['error_end']:]

            # Рахуємо score тексту
            score = score_text(text, model, vocab)
            scores.append(score)

            print(f'{score:.4f} {text}')

        # Сортуємо кандидатів на виправлення за score
        result = sorted(zip(scores, item['corrections']), key=lambda x: x[0])

        # Обираємо найкращу заміну
        best = result[0]
        best_word = best[1]
        choices.append(best_word)

        # Знаходимо чергову літеру повідомлення
        error = item['error']
        letter = get_letter(error, best_word)
        secret.append(letter)

    secret_message = ''.join(secret)

    return choices, secret_message

choices, secret_message = solve(model, vocab, lab.test_items)

lab.evaluate_accuracy(choices)
print("SECRET MESSAGE: ", secret_message)

5.5419 Бо хочеться спати і вам і менші.
4.9458 Бо хочеться спати і вам і міні.
3.9586 Бо хочеться спати і вам і мені.
4.6425 Бо хочеться спати і вам і пені.
4.2165 Бо хочеться спати і вам і гені.
5.8913 По всьому світу. Ти ж пік тином,
4.9968 По всьому світу. Ти ж під тином,
5.7850 По всьому світу. Ти ж лід тином,
6.0806 По всьому світу. Ти ж пі тином,
6.6141 З князями, темними грабами.
6.5357 З князями, темними рабами.
6.5643 З князями, темними рибами.
5.7420 З князями, темними разами.
7.3505 З князями, темними бабами.
5.4802 Сидить на призьбі коло хвати
5.9257 Сидить на призьбі коло хаби
4.8178 Сидить на призьбі коло хати
5.3113 Сидить на призьбі коло вати
13.2811 Щоб не кричали: “Ах! аїлах!
10.4802 Щоб не кричали: “Ах! аллах!
11.0809 Щоб не кричали: “Ах! галлах!
10.4123 Щоб не кричали: “Ах! аллахи!
11.2759 Щоб не кричали: “Ах! аллаха!
7.3675 Аж дивно, чудно. Гость устояв
7.6825 Аж дивно, чудно. Гость стяв
6.7565 Аж дивно, чудно. Гость стояв
6.0904 Аж дивно, чудно. Гость стояк
6.3916

In [ ]:
lab.answer("Ім'я Прізвище")

Відповідь неправильна 🛑
Ні, не так... 🤔


Відправте посилання на цей colab або PDF з ним на пошту oleksii.o.syvokon@lpnu.ua. Дякую!
